In [703]:
from typing import List, Optional
from pathlib import Path
from collections import defaultdict

In [704]:
import pandas as pd
import numpy as np

In [705]:
dataframes = sorted(Path('../results').glob('*.csv'))

In [706]:
key_columns = ['제목', '방송 시작']
columns = ['제목', '방송 기간','방송 시간', '방송 횟수', '장르', '채널', '방송 시작']

## 방송 시작 순 정렬 및 2000년도 이하 제거

In [707]:
def delete_under(frame):
    frame_drop = frame[frame["방송 시작"] == "unknown"].index
    frame = frame.drop(frame_drop)
    frame["방송 시작"] = frame["방송 시작"].astype('datetime64[ns]')
    frame_under = frame[frame["방송 시작"].dt.year < 2000].index
    frame = frame.drop(frame_under).reset_index(drop=True)
    return frame

In [708]:
#필요 Parsing array
yoil = ['월','화','수','목','금','토','일']
check = ['~']
is_afternoon = ["오전","오후"]

## 방송 시간 데이터 정리

In [709]:
def parsing_time(frame):
    temp = frame["방송 시간"]
    
    for index, value in enumerate(temp):
        if(temp[index] != temp[index]):
            continue
        temp[index] = temp[index].replace("요일","")
        temp[index] = temp[index].replace("문화방송","")
        temp[index] = temp[index].replace("평일","월금")
        temp[index] = temp[index].replace("주말","토일")
        temp[index] = temp[index].replace("밤","오후")
        temp[index] = temp[index].replace("저녁","오후")
        temp[index] = temp[index].replace("아침","오전")
            
#     frame = pd.merge(frame, temp, left_index = True, right_index = True, how='right')
                
    time_data = set_time(temp)
    date_data = parsing_dates(temp)
    
    time_pd = pd.DataFrame.from_dict(time_data, orient = 'index' ,columns = ["방송시작시간"])
    date_pd = pd.DataFrame.from_dict(date_data, orient = 'index' ,columns = yoil)
    
    time_first = pd.merge(frame, time_pd, left_index = True, right_index = True, how='right')
    result = pd.merge(time_first, date_pd, left_index = True, right_index = True, how='right')
    return result
    
    

## 방송 시작 시간 정리

In [710]:
def set_time(temp):
    time_date = {}
    time_arr = []
    is_hour_1 = re.compile('([0-9 ]+?)시')
    is_hour_2 = re.compile('([0-9 ]+?)\:')
    for index, value in enumerate(temp):
        if temp[index] != temp[index]:
            time_date[index] = [ ]        
        else:
            aa = is_hour_1.findall(temp[index])
            if not aa or aa[0] == " ":
                aa = is_hour_2.findall(temp[index])
                if not aa or aa[0] == " ":
                    time_date[index] = [ ] 
                    continue
            time_arr.append(int(aa[0]))
            for i in is_afternoon:
                if i in temp[index]:
                    if i == "오후":
                        time_arr[0] = time_arr[0] + 12
                    #time_arr.append(i)
                    break
            time_date[index] = tuple(time_arr)
            time_arr.clear()
            
    return time_date


## 요일의 One hot coding

In [711]:
def fill_one(arr):
    one_count = 0
    for index, value in enumerate(arr):
        if value == 0 and one_count == 1:
            arr[index] = 1
        elif value == 1 and one_count == 0:
            one_count = 1
        elif value == 1 and one_count == 1:
            one_count = 0   

In [712]:
def parsing_dates(temp):
    bool_date = {}
    arr = []
    naun_arr = []
    for index, value in enumerate(temp):
        for i in yoil:
            #Nan 값 제거
            if temp[index] != temp[index]:
                break
            if i in value:
                arr.append(1)
            else:
                arr.append(0)
        if temp[index] != temp[index]:
            #비어 있는 index 처리
            naun_arr.append(index)
            for seven_time in range(7):
                arr.append(0)
        #물결표시
        elif temp[index].find(check[0]):
            fill_one(arr)     
        bool_date[index] = tuple(arr)
        arr.clear()
    return bool_date

In [713]:
# frames = {
#     data.stem: pd.read_csv(str(data))
#     for data in dataframes
#     if data.stem != 'results'
# }

# for frame in frames.values():
#     assert (frame.columns == columns).all()
    
frames = {
    data.stem: pd.read_csv(str(data))
    for data in dataframes
    if data.stem != 'results'
}

for key, frame in frames.items():
    frames[key] = delete_under(frame)
    frames[key] = parsing_time(frame)
    assert (frame.columns == columns).all()

In [714]:
frames['namuwiki']
# frames['wikipedia']

,제목,방송 기간,방송 시간,방송 횟수,장르,채널,방송 시작,방송시작시간,월,화,수,목,금,토,일
0,그남자오수,2018년 3월 5일 ~ 2018년 4월 24일,"월, 화 21:00~22:00",16부작,로맨스 범죄,OCN,2018-03-05 00:00:00,21.0,1,1,0,0,0,0,0
1,간택-여인들의전쟁,2019년 12월 14일 ~ 2020년 2월 9일,"토, 일 오후 10:50 ~",16부작,"로맨스, 퓨전사극",TV CHOSUN,2019-12-14 00:00:00,22.0,0,0,0,0,0,1,1
2,우리가만난기적,2018년 4월 2일 ~ 2018년 5월 29일,"월, 화 오후 10:00 ~ 11:10",18부작,NaN,KBS 2TV,2018-04-02 00:00:00,22.0,1,1,0,0,0,0,0
3,왕과나,2007년 8월 27일 ~ 2008년 4월 1일,"월, 화 오후 9:55 ~ 11:05",63부작,NaN,SBS,2007-08-27 00:00:00,21.0,1,1,0,0,0,0,0
4,굿캐스팅,2020년 4월 ~ (예정),"월, 화 오후 10:00 ~ 11:10",,NaN,SBS,unknown,22.0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,모두다김치,2014년 4월 7일 ~ 2014년 10월 31일,월 ~ 금 오전 7시 50분,132부작,막장 드라마,문화방송,2014-04-07 00:00:00,7.0,1,1,1,1,1,0,0
732,49일,2011년 3월 16일 ~ 2011년 5월 19일,"수, 목 09:55 ~ 11:05",20부작,판타지 드라마,SBS,2011-03-16 00:00:00,9.0,0,0,1,1,0,0,0
733,로맨스타운,2011년 5월 11일 ~ 2011년 7월 14일,"수, 목 오후 9:55~11:05",20부작,로맨틱 코미디,KBS 2TV,2011-05-11 00:00:00,21.0,0,0,1,1,0,0,0
734,명랑소녀성공기,2002년 3월 13일 ~ 2002년 5월 2일,"수, 목 오후 9시 55분 ~ 오후 11시 5분",16부작,NaN,SBS,2002-03-13 00:00:00,21.0,0,0,1,1,0,0,0


In [715]:
total = defaultdict(set)

In [716]:
for key, frame in frames.items():
    for _, args in frame[key_columns].iterrows():
        total[tuple(args.values)].add(key)

## Merge multiple domain dataframes

In [717]:
def merge(values: List[str]) -> str:
    """Simple merge return maximum length data
    """
    def valuable(value: np.ndarray) -> int:
        item = value.item()
        if isinstance(item, str):
            return len(item)
        return 0
    
    index = np.array(tuple(map(valuable, values))).argmax()
    
    return values[index].item()

In [718]:
table = pd.DataFrame([], columns=columns)

In [719]:
for key, domains in total.items():
    if len(domains) > 1:
        table = table.append(pd.Series([merge([frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)][column].values for domain in domains]) for column in columns], index=columns), ignore_index=True)
    else:
        domain, *_ = domains
        table = table.append(frames[domain].loc[(frames[domain][key_columns] == key).all(axis=1)])

In [720]:
print(table.shape)
table.head()

(1910, 15)


,제목,방송 기간,방송 시간,방송 횟수,장르,채널,방송 시작,방송시작시간,월,화,수,목,금,토,일
0,그남자오수,2018년 3월 5일 ~ 2018년 4월 24일,"월, 화 21:00~22:00",16부작,로맨스 범죄,OCN,2018-03-05 00:00:00,21.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,간택-여인들의전쟁,2019년 12월 14일 ~ 2020년 2월 9일,"매주 토, 일 오후 11시 ~ 오전 0시",16부작,"로맨스, 퓨전사극",TV CHOSUN,2019-12-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,우리가만난기적,2018년 4월 2일 ~ 2018년 5월 29일,"매주 월, 화 오후 (시간) 10:00 ~ 11:10",18부작,"힐링, 판타지, 멜로, 가족, 로맨스",KBS 2TV,2018-04-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,왕과나,2007년 8월 27일 ~ 2008년 4월 1일,"월, 화 오후 9:55 ~ 11:05",63부작,대하사극,SBS,2007-08-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,굿캐스팅,2020년 4월 ~ (예정),"월, 화 오후 10:00 ~ 11:10",,NaN,SBS,unknown,22.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
table.to_csv('../results/results.csv', index=None)